<a href="https://colab.research.google.com/github/promptmule4real/promptmule_demo/blob/main/PromptMule_demo_v1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PromptMule API Demo Suite v1.1

# Welcome to the **PromptMule API Demo Suite**! This hands-on tool is designed to streamline your experience with the PromptMule API, allowing you to delve into various features, from generating dynamic prompts to assessing semantic response scores, and observing your token usage efficiency through our advanced caching mechanisms.

## What Does This Code Offer?

# This interactive script serves as your gateway to the PromptMule API, simplifying access to AI models through optimized API requests. With this suite, you gain the ability to:

# 1. Dynamically create and dispatch a series of prompts to the API, experimenting with various semantic similarities and response limits.
# 2. Monitor detailed logs of each API interaction (available in verbose mode) for in-depth analysis.
# 3. Compile comprehensive reports showcasing prompt responses, token utilization, and cache-enabled savings.

## Getting Started

# - **Running the Script**: Ensure you're in a compatible Python environment before initiating the script.
# - **API Key**: Enter your unique API key obtained from the PromptMule website (https://promptmule.com).
# - **Verbose Mode Selection**: Opt for verbose mode ("yes") to access detailed API communication logs or choose "no" for a more streamlined, less cluttered output.
# - **Cost-Saving Strategy Input**: Define your cost-saving strategy theme, forming the basis for the generated test prompts.

## Need More Information?

# Dive deeper into the intricate workings of the PromptMule API, explore our extensive [documentation](https://promptmule.com/docs), check out the [support discord](https://discord.gg/Ceun9EQ8) or reach out for support on the [PromptMule website](https://promptmule.com).

# Embark on your journey to efficient AI interactions. Happy testing!

# Demo Notebook

# Access the interactive demonstration Jupyter Notebook through this link: [PromptMule API Demo Notebook](https://github.com/promptmule4real/promptmule_demo/blob/main/promptmule_reporting_demo.ipynb).

# License

# This project is licensed under the MIT License - see the [LICENSE.md](https://github.com/promptmule4real/report_demo/blob/ce9f220dd37427538a564fddbdcb6ef122b1ea2f/LICENSE) file for details.


In [ ]:
# Required Libraries
import requests
import json
from IPython.display import display
import ipywidgets as widgets
import pandas as pd
import random

# Constants
BASE_URL = 'https://api.promptmule.com/'

# Constants for API interactions
ENDPOINT = 'https://api.promptmule.com/'
#LOGIN = 'login'
#KEY_GEN = 'api-keys'
PROMPT = 'prompt'
LIST_API_KEYS = 'api-keys/'
DELETE = 'profile/'
USAGE = 'usage'
DAILY_STATS = 'usage/daily-stats'

# Set display options for better readability
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)  # Show full width of the column content
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping of the dataframe


In [ ]:
from re import DEBUG

#api_key = " "
#print("Input your PromptMule API Key:")
#api_key = input()

# DEBUG
DEBUG = False

api_key = "6VDD3H6B4Y8Aftlnm7n069JvRIkaucmOIj1YyE69" # demo key
users_message = "improving user experience"
users_message = "improving profitability"
users_message = "reducing costs of OpenAI usage"

print(f"Your API key is: {api_key}")

In [ ]:
# Function to formulate a prompt message given a user's message and a variation number
def generate_prompt(users_message, variation):
    base_prompt = "You are a developer aiming to save money using OpenAI. Share only one implementation tip for using a cache-as-a-service wiht your code: "
    return f"{base_prompt}{users_message}. Variation: {variation}"

# Function to extract semantic scores and associated data from the API's response
def track_semantic_scores(response_json):
    return [(choice['prompt-id'], choice.get('score'), choice['message']['content'], choice['index']) for choice in response_json['choices']]

# Function to send a series of prompt requests to the API based on specified parameters
def send_prompt_requests(api_key, users_message, semantic_similarity, max_response_quantity, variations, df):
    headers = {"Content-Type": "application/json", "x-api-key": api_key}
    prompt_variations = variations

    for variation in range(1, int(prompt_variations * 0.9) + 1):  # Use 90% of the available variations
        api_call_body = {
            "model": "gpt-3.5-turbo",
            "messages": [{"role": "user", "content": generate_prompt(users_message, variation)}],
            "max_tokens": "50",
            "temperature": "0.2",
            "user": "demouser",
            "api": "openai", # This is specific to Promptmule and selects the LLM you'd like to prompt
            "semantic": str(semantic_similarity), # This determines the percentage prompt similarity that is acceptable to return from the cache for this call
            "sem_num": str(max_response_quantity) # This determines the max number of prompt/response pairs to return if they are greater than "semantic" percentage match
        }

        if DEBUG:
          print("api call body:", api_call_body)
          print(json.dumps(api_call_body, indent=4))
          print("\nSending prompt request:",json.dumps(api_call_body, indent=4))

        response = requests.post(ENDPOINT + PROMPT, headers=headers, json=api_call_body)

        if response.status_code == 200:
            if DEBUG:
              print("\nPrompt response:")
              print(json.dumps(response.json(), indent=4))
            else:
              print(".")

            choices_data = track_semantic_scores(response.json())
            for choice in choices_data:
                df.loc[len(df)] = [choice[0], semantic_similarity, choice[1], choice[2], choice[3]]
                if DEBUG:
                  print("\nTop Implementation Tip:", choices_data[0][2])
                  print("\nOther Tips:", *[choice[2] for choice in choices_data[1:]], sep="\n")
        else:
          print(f"\nError with prompt variation {variation}. Status code: {response.status_code}")
          print(json.dumps(response.json(), indent=4))

# Function to fetch data from an API endpoint and print the results
def fetch_data(headers, endpoint, success_message, fail_message):
    #headers = {"Content-Type": "application/json", "x-api-key": api_key}
    response = requests.get(endpoint, headers=headers)
    if response.status_code == 200:
        print(f"\n{success_message}:")
        print(json.dumps(response.json(), indent=4))
    else:
        print(f"\n{fail_message} {response.status_code}:")
        print(json.dumps(response.json(), indent=4))

In [ ]:
# --- MAIN EXECUTION ---

# 1. Gather user input for prompt testing
#def get_cost_strategy():
#    print("Please enter the most important development aspect for your app:")
#    users_message = input()
#
#    if not users_message:
#        print("A cost-saving strategy theme is required. Please try again.")
#        return get_cost_strategy()
#
#    return users_message

#users_message = get_cost_strategy()

# 2. Define constants for prompt variations and requests, and a dataframe for reporting
prompt_variations = 3
requests_per_variation = 3
df = pd.DataFrame(columns=['Prompt ID', 'Similarity Target', 'Similarity Score', 'Response', 'Response Index'])
data_frame_output = df

# 3. Send prompt requests with varying semantic similarities and max response counts
semantic_nums = [1.0, 0.75, 0.5]
max_responses = list(range(1, 11))  # From 1 to 10
variations = 3
i = 0
for sem_num in semantic_nums:
    i += 1
    send_prompt_requests(api_key, users_message, sem_num, random.choice(max_responses), variations, df)

In [ ]:
# 5. Display the results in a structured report
print("\nPrompt-ing Report:")
print("Prompt:", )
print(df.to_string(index=True))

In [ ]:
#username = input()
#pw = input()

username = "demo"
pw = "Abcd@1234!"

# Function to log in to the service and retrieve authentication token
def login_to_promptmule(api_key, username, pw):
    headers = {"Content-Type": "application/json", "x-api-key": api_key}
    api_call_body = {
        "username": username,
        "password": pw
    }
    response = requests.post(ENDPOINT + LOGIN, headers=headers, json=api_call_body)

    if response.status_code == 200:
        return response.json()["token"]
    else:
        print(f"Login failed: {response.status_code}")
        print(json.dumps(response.json(), indent=4))
        print("Try Again.")
        return ""

token = login_to_promptmule(api_key, username, pw)
headers = {"Content-Type": "application/json", "x-api-key": api_key, "Authorization": f"Bearer {token}"}#

fetch_data(headers, ENDPOINT + LIST_API_KEYS, "List of API keys", "Failed to list API keys")

In [ ]:
fetch_data(headers, ENDPOINT + USAGE, "Usage Summary", "Failed to fetch usage summary")

In [ ]:
fetch_data(headers, ENDPOINT + DAILY_STATS, "Daily Usage Stats", "Failed to fetch daily usage stats")


In [ ]:
print("\nReport Summary:")
print(f"API Key: {headers['x-api-key'].split(' ')[-1]}")
print(f"Number of Cost-saving Strategy Variations Sent: {prompt_variations}")
print(f"Number of Requests per Strategy Variation: {requests_per_variation}")